In [69]:
import pandas as pd
import json
from pprint import pprint
import random
import math

file_path = './Clinical_and_Other_Features.xlsx'
data = pd.read_excel(file_path, sheet_name='Data')

# print("Columns in the dataset:")
# print(data.columns)

race_and_ethnicity_col = data.columns[21]
tumor_characteristics_col = data.columns[23:48]
mri_findings_col = data.columns[48:53]

# Print
# try:
#     print(data[mri_findings_col])
# except KeyError as e:
#     print(f"KeyError: {e}. Please check if the column names match exactly.")

In [70]:
# first_row_descriptions = data.iloc[0]
# print(first_row_descriptions)

# Race and Ethnicity 21
'''{0 = N/A,1 = white,2 = black,3 = asian,4 = native,
5 = hispanic,6 = multi,7 = hawa,8 = amer indian}'''
race_mapping = {
    0: 'Unknown', 
    1: 'White',
    2: 'Black',
    3: 'Asian',
    4: 'Native American',
    5: 'Hispanic',
    6: 'Multiple Races',
    7: 'Native Hawaiian',
    8: 'American Indian'
}

# Tumor Characteristics 23-48
# ER, PR
tc_er_mapping = {0: 'Negative', 1: 'Positive'}
tc_pr_mapping = {0: 'Negative', 1: 'Positive'}
# HER2
tc_her2_mapping = {0: 'Negative', 1: 'Positive', 2: 'Borderline'}
# Molecular Subtype
'''{0 = luminal-like,1 = ER/PR pos, HER2 pos,2 = her2,3 = trip neg}'''
tc_molecular_subtype_mapping = {
    0: 'Luminal-like',
    1: 'ER/PR Positive, HER2 Positive', 
    2: 'HER2', 
    3: 'Triple Negative' 
}
# Oncotype score - Open
tc_oncotype_score_mapping = {
    'NA': 'Not Available'
}

# MRI Findings 48-52
# Multicentric/Multifocal {0 = no, 1 = yes}
mri_multicentric_mapping = {0: 'No', 1: 'Yes'}
# Contralateral Breast Involvement {0 = no, 1 = yes}
mri_contralateral_mapping = {0: 'No', 1: 'Yes'}
# Lymphadenopathy or Suspicious Nodes {0 = no, 1 = yes}
mri_lymphadenopathy_mapping = {0: 'No', 1: 'Yes'}
# Skin/Nipple Invovlement {0 = no, 1 = yes}
mri_skin_nipple_mapping = {0: 'No', 1: 'Yes'}
# Pec/Chest Involvement {0 = no, 1 = yes}
mri_pec_chest_mapping = {0: 'No', 1: 'Yes'}

# Race and Ethnicity
race_questions_variants = [
    "What is the patient's race and ethnicity?",
    "Can you tell me the patient's race and ethnicity?",
    "Please provide the patient's race and ethnicity."
]

# Tumor Characteristics (ER, PR, HER2, Molecular Subtype)
# ER
tc_er_questions = [
    "What is the patient's ER status?",
    "Can you tell me the patient's ER status?",
    "Please provide the patient's ER status."
]
# PR
tc_pr_questions = [
    "What is the patient's PR status?",
    "Can you tell me the patient's PR status?",
    "Please provide the patient's PR status."
]
# HER2
tc_her2_questions = [
    "What is the patient's HER2 status?",
    "Can you tell me the patient's HER2 status?",
    "Please provide the patient's HER2 status."
]
# Molecular Subtype
tc_molecular_subtype_questions = [
    "What is the patient's molecular subtype?",
    "Can you tell me the patient's molecular subtype?",
    "Please provide the patient's molecular subtype.",
    "Tell me the patient's molecular subtype."
]
# Oncotype score
tc_oncotype_score_questions = [
    "What is the patient's Oncotype score?",
    "Can you tell me the patient's Oncotype score?",
    "Please provide the patient's Oncotype score."
]

# MRI Findings
# Multicentric/Multifocal
mri_multicentric_questions = [
    "Is the tumor multicentric or multifocal?",
    "Can you tell me if the tumor is multicentric or multifocal?",
    "Please provide if the tumor is multicentric or multifocal."
]
# Contralateral Breast Involvement
mri_contralateral_questions = [
    "Is there contralateral breast involvement?",
    "Can you tell me if there is contralateral breast involvement?",
    "Please provide if there is contralateral breast involvement."
]
# Lymphadenopathy or Suspicious Nodes
mri_lymphadenopathy_questions = [
    "Is there lymphadenopathy or suspicious nodes?",
    "Can you tell me if there is lymphadenopathy or suspicious nodes?",
    "Please provide if there is lymphadenopathy or suspicious nodes."
]
# Skin/Nipple Invovlement
mri_skin_nipple_questions = [
    "Is there skin or nipple involvement?",
    "Can you tell me if there is skin or nipple involvement?",
    "Please provide if there is skin or nipple involvement."
]
# Pec/Chest Involvement
mri_pec_chest_questions = [
    "Is there pectoral or chest wall involvement?",
    "Can you tell me if there is pectoral or chest wall involvement?",
    "Please provide if there is pectoral or chest wall involvement."
]


In [71]:
def question_generator(qid, patient_id, image_path, user_prompt, gpt_response, answer_type, column_name, column_idx):
    question = {
        "qid": qid,
        "patient_id": patient_id,
        "image": image_path or "to-do",
        "bbox": [],
        "conversation": [
            {
                "from": "user",
                "value": user_prompt
            },
            {
                "from": "gpt",
                "value": gpt_response
            }
        ],
        "metadata":{
            "dataset": "Duke-Breast-Cancer-MRI",
            "location": "Breast",
            "base_type": "vqa",
            "answer_type": answer_type,
            "column_name": column_name,
            "column_idx": column_idx
        }
    }
    return question

def row_to_vqa_json(row, patient_id, qid):
    questions = []
    
    race_question = question_generator(
        qid,
        patient_id,
        "",
        random.choice(race_questions_variants),
        race_mapping.get(row[race_and_ethnicity_col], 'Unknown') + '.',
        "CLOSE",
        "Race and Ethnicity",
        21
    )
    questions.append(race_question)
    qid += 1

    tc_er_question = question_generator(
        qid,
        patient_id,
        "",
        random.choice(tc_er_questions),
        tc_er_mapping.get(row[tumor_characteristics_col[0]], 'Unknown') + '.',
        "CLOSE",
        "ER",
        23
    )
    questions.append(tc_er_question)
    qid += 1

    tc_pr_question = question_generator(
        qid,
        patient_id,
        "",
        random.choice(tc_pr_questions),
        tc_pr_mapping.get(row[tumor_characteristics_col[1]], 'Unknown') + '.',
        "CLOSE",
        "PR",
        24
    )
    questions.append(tc_pr_question)
    qid += 1

    tc_her2_question = question_generator(
        qid,
        patient_id,
        "",
        random.choice(tc_her2_questions),
        tc_her2_mapping.get(row[tumor_characteristics_col[2]], 'Unknown') + '.',
        "CLOSE",
        "HER2",
        25
    )
    questions.append(tc_her2_question)
    qid += 1

    tc_molecular_subtype_question = question_generator(
        qid,
        patient_id, 
        "",
        random.choice(tc_molecular_subtype_questions),
        tc_molecular_subtype_mapping.get(row[tumor_characteristics_col[3]], 'Unknown') + '.',
        "CLOSE",
        "Molecular Subtype",
        26
    )
    questions.append(tc_molecular_subtype_question)
    qid += 1

    tc_oncotype_score_question = question_generator(
        qid,
        patient_id,
        "",
        random.choice(tc_oncotype_score_questions),
        (str(row[tumor_characteristics_col[4]])
        if isinstance(row[tumor_characteristics_col[4]], (int, float)) and not math.isnan(row[tumor_characteristics_col[4]])
        else tc_oncotype_score_mapping.get(row[tumor_characteristics_col[4]], 'Unknown')) + '.',
        "OPEN",
        "Oncotype Score",
        27
    )
    questions.append(tc_oncotype_score_question)
    qid += 1
        

    # MRI Findings
    mri_multicentric_question = question_generator(
        qid,
        patient_id,
        "",
        random.choice(mri_multicentric_questions),
        mri_multicentric_mapping.get(row[mri_findings_col[0]], 'Unknown') + '.',
        "CLOSE",
        "Multicentric/Multifocal",
        48
    )
    questions.append(mri_multicentric_question)
    qid += 1

    mri_contralateral_question = question_generator(
        qid,
        patient_id,
        "",
        random.choice(mri_contralateral_questions),
        mri_contralateral_mapping.get(row[mri_findings_col[1]], 'Unknown') + '.',
        "CLOSE",
        "Contralateral Breast Involvement",
        49
    )
    questions.append(mri_contralateral_question)
    qid += 1

    mri_lymphadenopathy_question = question_generator(
        qid,
        patient_id,
        "",
        random.choice(mri_lymphadenopathy_questions),
        mri_lymphadenopathy_mapping.get(row[mri_findings_col[2]], 'Unknown') + '.',
        "CLOSE",
        "Lymphadenopathy or Suspicious Nodes",
        50
    )
    questions.append(mri_lymphadenopathy_question)
    qid += 1

    mri_skin_nipple_question = question_generator(
        qid,
        patient_id,
        "",
        random.choice(mri_skin_nipple_questions),
        mri_skin_nipple_mapping.get(row[mri_findings_col[3]], 'Unknown') + '.',
        "CLOSE",
        "Skin/Nipple Involvement",
        51
    )
    questions.append(mri_skin_nipple_question)
    qid += 1

    mri_pec_chest_question = question_generator(
        qid,
        patient_id,
        "",
        random.choice(mri_pec_chest_questions),
        mri_pec_chest_mapping.get(row[mri_findings_col[4]], 'Unknown') + '.',
        "CLOSE",
        "Pec/Chest Involvement",
        52
    )
    questions.append(mri_pec_chest_question)
    qid += 1

    return questions, qid


In [72]:
questions_list = []
global_qid = 1
for i in range(2, len(data)):
    patient_id = data.iloc[i][0]
    row = data.iloc[i]
    questions, qid = row_to_vqa_json(row, patient_id, global_qid)
    global_qid = qid
    questions_list.extend(questions)

# pprint(vqa_json_list)
output_path = './duke_vqa.json'
with open(output_path, 'w') as f:
    json.dump(questions_list, f, indent=4)
    print(f"Saved to {output_path}")

Saved to ./duke_vqa.json
